In [22]:
from pymongo import MongoClient
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from textblob import TextBlob
import spacy
from spacy import displacy
import en_core_web_sm
import speech_recognition as sr 
import pyaudio

In [23]:
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Mariets/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Mariets/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Mariets/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Traer los datos

In [24]:
client = MongoClient("mongodb://localhost/Game_of_Thrones") 
db = client.get_database()
collection = db['frases']

In [25]:
personajes = collection.distinct('Name')
personajes[:25]

[None,
 'a voice',
 'addam marbrand',
 'aemon',
 'aeron',
 'aerson',
 'ahsa',
 'all',
 'all three',
 'all together',
 'alliser',
 'alliser thorn',
 'alliser thorne',
 'alliser throne',
 'allister',
 'alton',
 'amory',
 'anguy',
 'announcer',
 'archers',
 'archmaester',
 'areo',
 'armory',
 'arthur',
 'arya stark']

In [26]:
query_tyrion = {"Name":"tyrion lannister"}
tyrion = list(collection.find(query_tyrion, {"Sentence": 1,"_id": 0}))
tyrion

[{'Sentence': 'Mmh. It is true what they say about the Northern girls.'},
 {'Sentence': 'I did hear something about that.'},
 {'Sentence': 'And the other brother?'},
 {'Sentence': "There's the pretty one. And there's the clever one."},
 {'Sentence': 'I hear he hates that nickname.'},
 {'Sentence': 'Clever girl.'},
 {'Sentence': 'Have you?'},
 {'Sentence': 'The gods gave me one blessing.'},
 {'Sentence': 'Should I explain to you the meaning of a closed door in a whorehouse, brother?'},
 {'Sentence': 'She has odd cravings, our sister.'},
 {'Sentence': "I'm sorry, I've begun the feast a bit early. And this is the first of many courses."},
 {'Sentence': 'Close the door!'},
 {'Sentence': "Your uncle's in the Night's Watch."},
 {'Sentence': "Preparing for a night with your family. I've always wanted to see the Wall."},
 {'Sentence': "My greatest accomplishment. You – you're Ned Stark's bastard, aren't you?"},
 {'Sentence': 'Did I offend you? Sorry. You are the bastard, though.'},
 {'Sentence

## Meter los diálogos en un dataframe

### Con Mongo

In [27]:
tyrion_frases = [i['Sentence'] for i in tyrion]
tyrion_frases

['Mmh. It is true what they say about the Northern girls.',
 'I did hear something about that.',
 'And the other brother?',
 "There's the pretty one. And there's the clever one.",
 'I hear he hates that nickname.',
 'Clever girl.',
 'Have you?',
 'The gods gave me one blessing.',
 'Should I explain to you the meaning of a closed door in a whorehouse, brother?',
 'She has odd cravings, our sister.',
 "I'm sorry, I've begun the feast a bit early. And this is the first of many courses.",
 'Close the door!',
 "Your uncle's in the Night's Watch.",
 "Preparing for a night with your family. I've always wanted to see the Wall.",
 "My greatest accomplishment. You – you're Ned Stark's bastard, aren't you?",
 'Did I offend you? Sorry. You are the bastard, though.',
 'And Lady Stark is not your mother. Making you a bastard. Let me give you some advice, bastard. Never forget what you are. The rest of the world will not. Wear it like armor. Then it can never be used to hurt you.',
 "All dwarves are 

In [28]:
tyrion_lannisterdf = pd.DataFrame(tyrion_frases, columns=["Tyrion"])
tyrion_lannisterdf.head()

,Tyrion
0,Mmh. It is true what they say about the Northe...
1,I did hear something about that.
2,And the other brother?
3,There's the pretty one. And there's the clever...
4,I hear he hates that nickname.


### Con Pandas

In [29]:
GOT_Data = pd.DataFrame(list(collection.find()))
GOT_Data.head()

,_id,Release Date,Season,Episode,Episode Title,Name,Sentence
0,5fc4d7f86b6146ead96b2897,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...
1,5fc4d7f86b6146ead96b2898,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...
2,5fc4d7f86b6146ead96b2899,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?
3,5fc4d7f86b6146ead96b289a,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.
4,5fc4d7f86b6146ead96b289b,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.


In [30]:
GOT_Data = GOT_Data[['Name', 'Sentence', 'Season']]
GOT_Data.head()

,Name,Sentence,Season
0,waymar royce,What do you expect? They're savages. One lot s...,Season 1
1,will,I've never seen wildlings do a thing like this...,Season 1
2,waymar royce,How close did you get?,Season 1
3,will,Close as any man would.,Season 1
4,gared,We should head back to the wall.,Season 1


In [44]:
#Solo quiero coger los diálogos de la Temporada 1
GOT_Data_T1=GOT_Data[GOT_Data.Season=='Season 1']

In [45]:
GOT_Data_T1.groupby(['Name'])
GOT_Data_T1

,Name,Sentence,Season
0,waymar royce,What do you expect? They're savages. One lot s...,Season 1
1,will,I've never seen wildlings do a thing like this...,Season 1
2,waymar royce,How close did you get?,Season 1
3,will,Close as any man would.,Season 1
4,gared,We should head back to the wall.,Season 1
...,...,...,...
3174,daenerys targaryen,ls that what you fear? You will be my khalasar...,Season 1
3175,daenerys targaryen,"Ser Jorah, bind this woman to the pyre. You sw...",Season 1
3176,mirri maz duur,You will not hear me scream.,Season 1
3177,daenerys targaryen,I will. But it is not your screams I want. Onl...,Season 1


## Sentiment analysis

In [46]:
sia = SentimentIntensityAnalyzer()

In [47]:
# la primera frase que dice Tyrion Lannister en la serie
sentence = "Mmh. It is true what they say about the Northern girls"

In [48]:
polarity = sia.polarity_scores(sentence)

In [49]:
polarity

{'neg': 0.0, 'neu': 0.781, 'pos': 0.219, 'compound': 0.4215}

In [50]:
def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    return polarity

In [51]:
def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    pol = polarity['compound']
    return pol

In [57]:
GOT_Data_T1['sentiment_compound'] = GOT_Data_T1['Sentence'].apply(sentimentAnalysis)
GOT_Data_T1.head()

<ipython-input-57-75002a1219cd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GOT_Data_T1['sentiment_compound'] = GOT_Data_T1['Sentence'].apply(sentimentAnalysis)


,Name,Sentence,Season,sentiment_compound
0,waymar royce,What do you expect? They're savages. One lot s...,Season 1,-0.7717
1,will,I've never seen wildlings do a thing like this...,Season 1,0.6124
2,waymar royce,How close did you get?,Season 1,0.0000
3,will,Close as any man would.,Season 1,0.0000
4,gared,We should head back to the wall.,Season 1,0.0000


In [60]:
GOT_Data_T1.groupby(['Name'])['sentiment_compound'].mean()

Name
a voice           0.000000
addam marbrand    0.210750
all               0.000000
alliser thorne    0.127654
arya stark       -0.053591
                    ...   
waymar royce     -0.385850
wildling         -0.260086
will             -0.091486
wine merchant     0.053864
yoren            -0.079978
Name: sentiment_compound, Length: 146, dtype: float64